# Hidden variables

In [1]:
# No "private" variables, but we can "hide" them
class Test():
    def __init__(self, name = "bob") -> None:
        self.__test = 'hello'
        self.name = name


test = Test()
print(dir(test)) # pay special attention to the dir listing here.  What do you see in the first element??
try:
    print(test.__test) # this doesn't work because it's 'hidden'
except AttributeError:
    print('caught attribute error')
print(test._Test__test) # this works due to "name mangling"

['_Test__test', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__', 'name']
caught attribute error
hello


What about attributes in parent and child classes??

In [2]:
class Parent():
    def __init__(self) -> None:
        self.__test = 'hello parent'

class Child(Parent):
    def __init__(self) -> None:
        super().__init__()
        self.__test = 'hello child'
        self._Parent__test = 'test 3' # you can overwrite the parent hidden variable...

child = Child()
print(dir(child))
print(child._Child__test)
print(child._Parent__test)

['_Child__test', '_Parent__test', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__']
hello child
test 3


In [7]:
# Another one with a "has a" relationship

class OtherClass():
    def __init__(self) -> None:
        self.__test = 'help me'

class HasAClass():
    def __init__(self) -> None:
        self.__test = 'has a'
        self.__hidden = OtherClass() # instantiate a new object

test = HasAClass()
print(dir(test))
print(test._HasAClass__hidden._OtherClass__test)

['_HasAClass__hidden', '_HasAClass__test', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__']
help me


# Overriding methods

In [6]:
# Test
class Test():
    def __init__(self, name) -> None:
        self.name = name

    def __add__(self, other):
        print(f"Did the add thing from {self.name} to {other.name}")
    
class Test2():
    def __init__(self, name) -> None:
        self.name = name

class Test3():
    def __init__(self) -> None:
        pass # no name attribute here

alice = Test("alice")
bob = Test2("bob")
charlie = Test3()

alice + bob # Note that this is valid even though Bob has a different class!!!!!!!
alice.__add__(bob) # this is how the interpreter actually executes the line above
# `alice` becomes self, `bob` becomes other (or whatever the argument is named)

try:
    print(bob + alice)
except TypeError:
    print("Cannot add from Test2 to Test because __add__ is not defined on bob")

try:
    alice + charlie
except AttributeError:
    print("This is a different error because charlie doesn't have a .name attribute")


Did the add thing from alice to bob
Did the add thing from alice to bob
Cannot add from Test2 to Test because __add__ is not defined on bob
This is a different error because charlie doesn't have a .name attribute


# Dirty deeds...

In [ ]:
class NewTest():
    def __init__(self) -> None:
        self.stuff = "my_name"

    def __str__(self) -> str:
        return self.stuff

    def __add__(self, other):
        return self.__str__() + " " + other
    
my_obj = NewTest()
print(my_obj + 'test') # this is normal and expected

# you can always rewrite to
# my_obj.__add__('test')
# and see if it makes sense based on your method definition

try:
    print('test' + my_obj)
except TypeError:
    print('this one fails because string cannot add a NewTest object to it')

NewTest.__radd__ = lambda self, other: other.__str__() + " " + self.__str__()  # here I am adding __radd__ as a method dynamically
# please please please never do this ^^^

print('test' + my_obj) # this uses the __radd__ method (right-add), now it works!


# The better way to define the method...
# class NewTest():
#     def __init__(self) -> None:
#         self.stuff = "my_name"

#     def __str__(self) -> str:
#         return self.stuff

#     def __add__(self, other):
#         return self.__str__() + " " + other
    
#     def __radd__(self, other):
#         return other.__str__() + " " + self.__str__()



my_name test
this one fails because string cannot add a NewTest object to it
test my_name
